<a href="https://colab.research.google.com/github/PrettyCharity/Bigquery/blob/main/San_Francisco_Bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Google Cloud Credentials
import os
from google.cloud import bigquery
# Credentials and the path
credential_path = "key.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

In [2]:
#@title Accessing the 'san_francisco' via bigquery object
client = bigquery.Client()
dataset_ref = client.dataset('san_francisco', project = 'bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

In [3]:
#@title Exploring the tables of san_francisco
tables = client.list_tables(dataset)
for table in tables:
  print(table.table_id)

311_service_requests
bikeshare_stations
bikeshare_status
bikeshare_trips
film_locations
sffd_service_calls
sfpd_incidents
street_trees


In [4]:
#@title Obtaining 'bikeshare_trips' table
table_ref = dataset_ref.table('bikeshare_trips')
table = client.get_table(table_ref)
preview = client.list_rows(table, max_results = 5).to_dataframe()
print('Table columns:\n\n')
print(preview.columns)

Table columns:


Index(['trip_id', 'duration_sec', 'start_date', 'start_station_name',
       'start_station_id', 'end_date', 'end_station_name', 'end_station_id',
       'bike_number', 'zip_code', 'subscriber_type'],
      dtype='object')


In [15]:
#@title Construct the SQL query to find the (cumulative) number of trips per day in 2015
num_trips_query = """
                  WITH trips_by_day AS
                  (
                      SELECT DATE(start_date) AS trip_date, 
                          COUNT(*) AS num_trips
                      FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                      WHERE EXTRACT(YEAR FROM start_date) = 2015
                      GROUP BY trip_date
                  )
                  SELECT *,
                      SUM(num_trips)
                          OVER (
                                ORDER BY trip_date
                                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                               ) AS cumulative_trips
                  FROM trips_by_day
                  """


In [17]:
#@title Fetch data from the server and show the results
# API request - run the query, and convert the results to a pandas DataFrame
query_job = client.query(num_trips_query)
num_trips = query_job.to_dataframe()

num_trips.head(10)

,trip_date,num_trips,cumulative_trips
0,2015-01-01,181,181
1,2015-01-02,428,609
2,2015-01-03,283,892
3,2015-01-04,206,1098
4,2015-01-05,1186,2284
5,2015-01-06,1255,3539
6,2015-01-07,1301,4840
7,2015-01-08,1295,6135
8,2015-01-09,1173,7308
9,2015-01-10,362,7670


In [18]:
#@title Construct the SQL query to to track beginning and ending stations on Augustus 4, 2015, for each bike
start_end_query = """
                  SELECT bike_number,
                      TIME(start_date) AS trip_time,
                      FIRST_VALUE(start_station_id)
                          OVER (
                                PARTITION BY bike_number
                                ORDER BY start_date
                                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                                ) AS first_station_id,
                      LAST_VALUE(end_station_id)
                          OVER (
                                PARTITION BY bike_number
                                ORDER BY start_date
                                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                                ) as last_station_id,
                     start_station_id,
                     end_station_id
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE DATE(start_date) = '2015-08-04'
                  """


In [20]:
#@title Fetch data from the server and show the results
# API request - run the query, and convert the results to a pandas DataFrame
query_job = client.query(start_end_query)
start_end = query_job.to_dataframe()

start_end.head(10)

,bike_number,trip_time,first_station_id,last_station_id,start_station_id,end_station_id
0,59,15:52:00,23,25,23,25
1,556,09:27:00,62,70,62,70
2,556,09:38:00,62,70,70,51
3,556,16:52:00,62,70,51,70
4,590,17:29:00,64,59,64,51
5,590,18:13:00,64,59,51,64
6,590,18:15:00,64,59,64,77
7,590,18:33:00,64,59,77,59
8,318,07:32:00,50,55,50,68
9,318,17:09:00,50,55,68,69
